In [ ]:
string = ' ciao mi ✅ riccardo :) '

print(string)

In [ ]:
import pandas as pd 
tweets_df = None 
pd.Series(' '.join(tweets_df[tweets_df['label']=='1']['tweet']).lower().split()).value_counts()[:1000].to_dict().keys()

In [ ]:
import reticker 
# reticker.config.BLACKLIST.add("RT")
config = reticker.TickerMatchConfig(unprefixed_uppercase=False,prefixed_titlecase=False)
extractor = reticker.TickerExtractor(match_config=config)

df1 = tweets_df[tweets_df['tweet'].str.contains('(?<!\S)\$[a-zA-Z]+(?!\S)', regex=True)]
df3 = tweets_df[tweets_df['tweet'].str.contains('[$][A-Za-z][\S]', regex=True)]
df4 = tweets_df[tweets_df['tweet'].str.contains(extractor.pattern, regex=True)]



In [ ]:
l = tweets_df['tweet'].str.findall("(?<!\S)\$[0-9]+(?!\S)")

lb = [el for el in l if el]

l2 = tweets_df['tweet'].str.findall("\\B\$[0-9]+(?:\.\d+)?[k+]?")

lb2 = [el for el in l2 if el]

import itertools
lb = list(itertools.chain(*lb))
lb2 = list(itertools.chain(*lb2))

print(list(set(lb2) - set(lb)))

In [ ]:
EMOTICONS = [':\)','\(:',':-\)','\s:\^\)\s','\s=\]\s','\s:-\(\s','\s:\(\s','\s:\(\s','\^_\^','\s:\'\(\s','\s:-D\s']